# DigiB Assignment

In [1]:
spark

## Testing Connection to Kafka Topic

In [1]:
!pip install kafka-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.5/246.5 KB 5.6 MB/s eta 0:00:00a 0:00:01
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [2]:
from kafka import KafkaConsumer

bootstrap_servers = ['kafka:9092']
topicName = 'orders'

consumer = KafkaConsumer(
    topicName, 
    group_id='group1', 
    bootstrap_servers=bootstrap_servers)

In [3]:
# Read and print message from consumer
counter = 3
for msg in consumer:
    if counter == 0:
        break
    print("Topic Name=%s,Message=%s"%(msg.topic,msg.value))
    counter = counter - 1

Topic Name=orders,Message=b'{"order_id": "efdd15dd-0737-404a-93cc-82262e09f0d7", "customer_id": "b3b191a0-745b-4e14-b74c-ba2cf6b30526", "order_lines": [{"product_id": "6fd457c1-a7ef-4a5a-8e00-ebce44726f8f", "volume": 98, "price": 30.23}, {"product_id": "585c890e-bb05-48a4-a8a8-20b5b569296b", "volume": 87, "price": 69.0}, {"product_id": "c3a9799e-c666-4509-9d81-08015bcd2628", "volume": 63, "price": 81.86}, {"product_id": "a92f5327-c987-479d-9470-f1239916390a", "volume": 31, "price": 93.53}, {"product_id": "9772ffc1-3fdf-4187-a9cf-519f334475c2", "volume": 79, "price": 25.91}, {"product_id": "65201884-97f3-4c18-90c3-b42d5b277e97", "volume": 14, "price": 74.68}, {"product_id": "17b72146-fb11-422c-863b-1cb6e0a2f852", "volume": 61, "price": 36.5}, {"product_id": "92252cf5-cb90-45a2-bb41-92aea8fb9e2c", "volume": 50, "price": 79.69}, {"product_id": "cf582135-2855-42e2-9580-e26294efc1a3", "volume": 21, "price": 29.04}, {"product_id": "2464bf0d-133a-4ec5-ae96-4a86f5a657d1", "volume": 54, "price"

Topic Name=orders,Message=b'{"order_id": "56081dec-b5cf-4fd0-8a14-f17bea553fda", "customer_id": "a29eb877-77f7-475c-8e21-aef3c86bb975", "order_lines": [{"product_id": "bde8a4c0-cca1-49b2-9a10-739ece35df0e", "volume": 38, "price": 88.74}, {"product_id": "28534a1c-532e-4825-83bc-a313c766b62b", "volume": 55, "price": 43.79}, {"product_id": "2264ae86-4d93-405d-8b89-cdfad7fe5145", "volume": 9, "price": 75.08}, {"product_id": "9fe7dbc7-e2a5-4f78-b2e3-989889c3e147", "volume": 99, "price": 19.74}, {"product_id": "462715fd-3948-4131-a623-1487395a82fc", "volume": 33, "price": 16.29}, {"product_id": "c72392d1-ac64-4e84-a441-d33d98fde34a", "volume": 89, "price": 39.15}, {"product_id": "d8b81745-85ec-445f-ae6e-ec78b9c43198", "volume": 6, "price": 36.7}, {"product_id": "a2775759-2aec-4512-a7b7-b41ed47e3aee", "volume": 93, "price": 8.73}, {"product_id": "76926074-8ae8-4a99-8c2c-e07308b0b2b7", "volume": 47, "price": 61.46}, {"product_id": "efa1ed4b-06cb-4a4d-91e5-b54f5e60475c", "volume": 13, "price": 

## Functions for Data Processing

In [4]:
# Importing necessary libraries
from pyspark.sql import DataFrame, Window, SparkSession
import pyspark.sql.functions as fs

# Setting variables
NS = "dal"

### Utility Functions

In [5]:
def deduplicate(df, partition_key, order_column):
    
    window = Window.partitionBy(*partition_key).orderBy(order_column)
    return (
        df.withColumn("RowNr", fs.row_number().over(window)).where("RowNr = 1").drop("RowNr")
    )

def check_for_table(spark: SparkSession, model, table_name):
    
    try:
        spark.read.table(f"{model}.{table_name}")
        return True        
    except:
        pass

### Database Object Functions

In [6]:
def create_iceberg_table(spark: SparkSession, db_name, table_name, columns, replace: bool = False):
    
    try:
        if replace:
            spark.sql(f"CREATE OR REPLACE TABLE {NS}.{db_name}.{table_name} ({columns}) USING iceberg")
            print(f"Table {table_name} replaced.")
        else:
            spark.sql(f"CREATE TABLE IF NOT EXISTS {NS}.{db_name}.{table_name} ({columns}) USING iceberg")
            print(f"Table {table_name} created.")
            
    except Exception as e:
        print(f"Table creation failed with error {e}")
        
def append_to_iceberg_table(spark: SparkSession, df, model, table_name):
    
    try:
        if not check_for_table(spark, model, table_name):
            print(f"Table {table_name} does not exist.")
        df.writeTo(f"{NS}.{model}.{table_name}").append()
        print(f"Data appended to table {table_name}.")
    except Exception as e:
        print(f"Failed to append to table {table_name} with error {e}")        
        
def read_minio_data(spark: SparkSession, source_bucket, file_name, is_json = False):
    
    inputPath = f"s3a://{source_bucket}/{file_name}"
    
    if is_json:
        return spark.read.json(inputPath).drop("order_lines")
    else:
        return spark.read.option("header", "true").format("s3selectCSV").csv(inputPath)   

### Database creation

In [7]:
def create_database(spark: SparkSession, db_name: str):
    
    try:
        spark.sql(f"CREATE DATABASE IF NOT EXISTS {NS}.{db_name} LOCATION 's3a://{NS}/{db_name}'")
        print(f"Database {db_name} created.")
    except Exception as e:
        print(f"Database creation failed with error {e}")

### Data Transformation

In [8]:
def process_customer_data(spark: SparkSession, df):
    
    df = (spark.sql("select * from stg.stg_Customer")
        .withColumnRenamed("customer_id", "CustomerId")
        .withColumnRenamed("company_name", "CompanyName")
        .withColumnRenamed("specialized_industries", "SpecializedIndustries"))
    
    return deduplicate(df, ["CustomerId"], fs.col("CompanyName"))
    
def process_industry_data(spark: SparkSession, df):
    
    df = (spark.sql("select * from stg.stg_Industry")
        .withColumnRenamed("industry", "Industry"))
    
    return deduplicate(df, ["Industry"], fs.col("Industry"))
    
def process_order_data(spark: SparkSession, df):
    
    df = (spark.sql("select * from stg.stg_StreamOrders")
        .withColumnRenamed("order_id", "OrderId")
        .withColumnRenamed("customer_id", "CustomerId")
        .withColumnRenamed("amoung", "Amount")
        .withColumnRenamed("timestamp", "Timestamp"))
    
    return deduplicate(df, ["OrderId"], fs.col("Timestamp"))
    
def process_product_data(spark: SparkSession, df):
    
    df = (spark.sql("select * from stg.stg_Product")
        .withColumnRenamed("product_id", "ProductId")
        .withColumnRenamed("product_name", "ProductName")
        .withColumnRenamed("price", "Price"))
    
    return deduplicate(df, ["ProductId"], fs.col("ProductName"))

In [9]:
def create_db(spark: SparkSession):
    
    create_database(spark, "stg")
    create_database(spark, "prd")
    
def create_tables(spark: SparkSession):
    
    create_iceberg_table(spark, "stg","stg_Product", "product_id string, product_name string, price string")
    create_iceberg_table(spark, "stg","stg_StreamOrders", "order_id string, customer_id string, amount double, timestamp string")
    create_iceberg_table(spark, "stg","stg_Customer", "customer_id string, company_name string, specialized_industries string")
    create_iceberg_table(spark, "stg","stg_Industry", "industry string")
    
    
def insert_into_product_staging(spark: SparkSession):
    
    df = read_minio_data(spark, "demo-data", "Products.csv")
    append_to_iceberg_table(spark, df, "stg", "stg_Product")

    
def insert_into_order_staging(spark: SparkSession):
    
    df = read_minio_data(spark, "demo-data", "orders.json", True)
    append_to_iceberg_table(spark, df, "stg", "stg_StreamOrders")

    
def insert_into_customer_staging(spark: SparkSession):
    
    df = read_minio_data(spark, "demo-data", "Customers.csv")
    append_to_iceberg_table(spark, df, "stg", "stg_Customer")
  
    
def insert_into_industry_staging(spark: SparkSession):
    
    df = read_minio_data(spark, "demo-data", "Industries.csv")
    append_to_iceberg_table(spark, df, "stg", "stg_Industry")
    

def product_stage_to_main(spark: SparkSession, replace: bool):
    
    df = spark.sql("select * from stg.stg_Product")
    df = process_product(spark, df)
    create_iceberg_table(spark, "prd", "Product", "ProductId string, ProductName string, Price double", replace=replace)
    append_to_iceberg_table(spark, df, "prd", "Product")
    

def order_stage_to_main(spark: SparkSession, replace: bool):
    
    df = spark.sql("select * from stg.stg_StreamOrders")
    df = process_order(spark, df)
    create_iceberg_table(spark, "prd", "StreamOrders", "OrderId string, CustomerId string, Amount double, Timestamp timestamp", replace=replace)
    append_to_iceberg_table(spark, df, "prd", "StreamOrders")
    
    
def industry_stage_to_main(spark: SparkSession, replace: bool):
    
    df = spark.sql("select * from stg.stg_Industry")
    df = process_industry(spark, df)
    create_iceberg_table(spark, "prd", "Industry", "Industry string", replace=replace)
    append_to_iceberg_table(spark, df, "prd", "Industry")
    
    
def customer_stage_to_main(spark: SparkSession, replace: bool):
    
    df = spark.sql("select * from stg.stg_Customer")
    df = process_customer(spark, df)
    create_iceberg_table(spark, "prd", "Customer", "CustomerId string, CompanyName string, SpecializedIndustries string", replace=replace)
    append_to_iceberg_table(spark, df, "prd", "Customer")

### Assignment Run

In [10]:
create_db(spark)
create_tables(spark)

22/10/28 14:32:16 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
Database stg created.
Database prd created.
Table stg_Product created.
Table stg_StreamOrders created.
Table stg_Customer created.
Table stg_Industry created.


In [11]:
insert_into_product_staging(spark)
insert_into_order_staging(spark)
insert_into_customer_staging(spark)
insert_into_industry_staging(spark)

Data appended to table stg_Product.


Data appended to table stg_StreamOrders.
Data appended to table stg_Customer.
Data appended to table stg_Industry.
